In [6]:
import pandas as pd
import os 
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Read the dataset
df = pd.read_csv(os.path.join(parent_dir, 'dataset', 'clean_data.csv'))

In [4]:
def clean_route(df):
    """
    Clean the 'route' column of the DataFrame and split it into 'origin', 'destination', and 'transit' columns.

    Parameters:
    df (DataFrame): DataFrame containing a 'route' column.

    Returns:
    DataFrame: DataFrame with 'origin', 'destination', and 'transit' columns.
    """
    df = df.dropna(subset=['route'])
    # Function to split the route into origin, destination, and transit
    def split_route(route):
        parts = route.split(' to ')
        origin = parts[0]
        if len(parts) > 1:
            destination, transit = parts[1].split(' via ') if ' via ' in parts[1] else (parts[1], None)
        else:
            destination, transit = None, None
        return origin, destination, transit

    # Apply the function to create new columns
    df[['origin', 'destination', 'transit']] = df['route'].apply(split_route).apply(pd.Series)
    # Replace 'LHR' and 'Heathrow' with 'London Heathrow'
    df['origin'] = df['origin'].replace({'LHR': 'London Heathrow', 'Heathrow': 'London Heathrow'})
    df['destination'] = df['destination'].replace({'LHR': 'London Heathrow', 'Heathrow': 'London Heathrow'})
    df['transit'] = df['transit'].replace({'LHR': 'London Heathrow', 'Heathrow': 'London Heathrow'})
    return df

In [7]:
df = clean_route(df)

In [8]:
df.head()

,id,date_review,day_review,month_review,month_review_num,year_review,verified,name,month_fly,month_fly_num,...,cabit_serv,food,ground_service,wifi,money_value,recommended,review,origin,destination,transit
0,0,2024-03-14,14,March,3,2024,True,E Carmere,March,3.0,...,4.0,3.0,3.0,NaN,3,yes,Boarding was difficult caused by vast majority...,London Heathrow,Brussels,None
1,1,2024-03-14,14,March,3,2024,True,E Carmere,March,3.0,...,3.0,2.0,1.0,NaN,2,no,Boarding started with a delay of some 20 minut...,Barbados,London heathrow,None
2,2,2024-03-12,12,March,3,2024,True,P Marten,February,2.0,...,1.0,2.0,1.0,NaN,1,no,Absolutely horrible customer service - will ne...,Toronto,Mumbai,London
3,3,2024-03-11,11,March,3,2024,False,S Mahoney,February,2.0,...,4.0,1.0,2.0,NaN,5,yes,BA is not what it used to be! As much as I lik...,Copenhagen,Port of Spain,London
4,4,2024-03-10,10,March,3,2024,True,Frank Fehle,March,3.0,...,1.0,1.0,3.0,3.0,1,no,"BA First, it's not even the best business clas...",Los Angeles,London,None


In [9]:
def split_aircraft_column(df):
    # Split the 'aircraft' column by '/', then by '-', ',', and '&'
    split_aircraft = df['aircraft'].str.split('/|-|,|&', expand=True)
    
    # Rename the columns
    split_aircraft.columns = [f'aircraft_{i+1}' for i in range(split_aircraft.shape[1])]
    
    split_aircraft = split_aircraft[['aircraft_1','aircraft_2']]
    # Concatenate the split columns with the original DataFrame
    df = pd.concat([df, split_aircraft], axis=1)
    
    return df

In [10]:
df = split_aircraft_column(df)

In [11]:
df.head()

,id,date_review,day_review,month_review,month_review_num,year_review,verified,name,month_fly,month_fly_num,...,ground_service,wifi,money_value,recommended,review,origin,destination,transit,aircraft_1,aircraft_2
0,0,2024-03-14,14,March,3,2024,True,E Carmere,March,3.0,...,3.0,NaN,3,yes,Boarding was difficult caused by vast majority...,London Heathrow,Brussels,None,A320,None
1,1,2024-03-14,14,March,3,2024,True,E Carmere,March,3.0,...,1.0,NaN,2,no,Boarding started with a delay of some 20 minut...,Barbados,London heathrow,None,Boeing 777,None
2,2,2024-03-12,12,March,3,2024,True,P Marten,February,2.0,...,1.0,NaN,1,no,Absolutely horrible customer service - will ne...,Toronto,Mumbai,London,NaN,NaN
3,3,2024-03-11,11,March,3,2024,False,S Mahoney,February,2.0,...,2.0,NaN,5,yes,BA is not what it used to be! As much as I lik...,Copenhagen,Port of Spain,London,NaN,NaN
4,4,2024-03-10,10,March,3,2024,True,Frank Fehle,March,3.0,...,3.0,3.0,1,no,"BA First, it's not even the best business clas...",Los Angeles,London,None,Boeing 777,300ER


In [12]:
def clean_aircraft(df, column_name):
    df[column_name] = df[column_name].str.replace(r'(?i)Boeing (\d+)', r'B\1', regex=True)
    df[column_name] = df[column_name].str.replace(r'(?i)777', 'B777')
    df[column_name] = df[column_name].str.replace(r'(?i)A(\d+)', r'A\1', regex=True)
    df[column_name] = df[column_name].str.replace(r'(?i)170', 'E170')
    df[column_name] = df[column_name].str.replace(r'(?i)190', 'E190')
    
    # # Extract 'A___' if present, else keep the same
    # df[column_name] = np.where(df[column_name].str.contains(r'(?i)A\d+'), df[column_name].str.extract(r'(?i)(A\d+)'), df[column_name])
    df[column_name] = df[column_name].str.extract(r'(?i)([A-Z]\d+)', expand=False)
    
    return df

In [13]:
df = clean_aircraft(df, "aircraft_1")
df = clean_aircraft(df, "aircraft_2")

In [16]:
df[["aircraft","aircraft_1","aircraft_2"]]

,aircraft,aircraft_1,aircraft_2
0,A320,A320,None
1,Boeing 777,B777,None
2,NaN,NaN,None
3,NaN,NaN,None
4,Boeing 777-300ER,B777,None
...,...,...,...
2992,A321,A321,None
2994,Boeing 747-400,B747,None
2995,Boeing 747-400,B747,None
2996,NaN,NaN,None
